In [ ]:
# python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
# source elbaff_iesta_venv/bin/activate

%pip show nlpaf


In [ ]:
from nlpaf.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np

import iesta.properties as prop  
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY


In [ ]:

liberal_each_dataloader = IESTAData(ideology=prop.LIBERAL_IDEOLOGY, methodology=METHODOLOGY.EACH)
liberal_training_each_df, liberal_data_path = liberal_each_dataloader.get_training_data()
liberal_each_dataloader.pivot_df


In [ ]:
liberal_each_dataloader.pivot_binary_effect

In [ ]:
liberal_training_each_df.columns.tolist()

In [ ]:
from iesta.machine_learning.feature_extraction import extract_features
extract_features(ideology="liberal", 
                        data_path = liberal_data_path, 
                        batch_size=1000, 
                        spacy_n_processors=3,
                        transformer_based_features= False,
                        )

In [ ]:
extract_features(ideology="liberal", 
                        data_path = liberal_data_path, 
                        batch_size=100, 
                        spacy_n_processors=1,
                        transformer_based_features= True,
                        )